In [1]:
import networkx as nx
import freeman as fm
import matplotlib.pyplot as plt
from pprint import pprint
import pymysql
import sys
from collections import defaultdict

In [2]:
from setup import mysql_setup
conn = mysql_setup()

In [3]:
anos = [['1940-01-01', '1949-12-31'], ['1950-01-01', '1959-12-31'],
        ['1960-01-01', '1969-12-31'], ['1970-01-01', '1979-12-31'],
        ['1980-01-01', '1989-12-31'], ['1990-01-01', '1999-12-31'],
        ['2000-01-01', '2009-12-31'], ['2010-01-01', '2019-12-31']]

In [35]:
def get_infos(dates):
    with conn.cursor() as cursor:
        try:
            cursor.execute('''
                SELECT
                    t.track_name, pl.playlist_name
                FROM
                    Track t 
                    INNER JOIN Album_Track altr USING (id_track)
                    INNER JOIN Album al USING (id_album)
                    INNER JOIN Playlist_Track pltr USING (id_track)
                    INNER JOIN Playlist pl USING (id_playlist)
                WHERE
                    al.release_date >= %s and al.release_date <= %s
                ''', (dates[0], dates[1]))
            ret = cursor.fetchall()
        except pymysql.err.IntegrityError as e:
            print("Erro: não foi possivel dar select em playlist")
            return None
    return ret

In [62]:
def faz_grafo(dic, ligacoes):
    
    g = nx.Graph()
    
    for ent in dic:
        g.add_node(ent)
        
    keys = list(dic.keys())
        
    for i in range(len(dic)-1): #musicas
        for pl1 in dic[keys[i]]: #playlists
            for j in range(len(dic)-1-i): #musicas
                count = 0
                for pl2 in dic[keys[j+1+i]]: #playlists
                    if pl1 == pl2:
                        count += 1
                if count >= ligacoes:
                    g.add_edge(keys[i], keys[j+1+i])
    
    nodes_to_remove = []
    
    for node in g.nodes:
        if len(g.edges(node)) < 1:
            nodes_to_remove.append(node)
            
    for node in nodes_to_remove:
        g.remove_node(node)
    
    return g

In [63]:
for ano in anos:
    infos = get_infos(ano)
    d = defaultdict(list)
    for info in infos:
        d[info[0]].append(info[1])
    break

In [148]:
infos = get_infos(['2010-01-01', '2019-12-31'])
d = defaultdict(list)
for info in infos:
    d[info[0]].append(info[1])

pprint(d)

In [149]:
g = faz_grafo(d, 1)

In [150]:
nx.write_gml(g, "test.gml")

In [151]:
g = fm.load('test.gml')

In [ ]:
g.set_all_nodes('labpos', 'hover')
g.move('kamada_kawai')
g.interact()

In [ ]:
g.draw()